In [1]:
using Revise
using StaticArrays: SVector, SMatrix, FieldVector
using LinearAlgebra
using Distributions
using BenchmarkTools
using Test

# Experiment on Types that will be used in Scattering.jl

## Testing Vectors

In [ ]:
const RVector{T} = SVector{3, T}
const RVectorList{T} = Vector{RVector{T}}

In [ ]:
r1 = RVector(1, 2, 3.0)
r2 = RVector(3.0, 2, 1)

In [ ]:
struct Point{T <: Real} <: FieldVector{3, T}
    x::T
    y::T
    z::T
end

In [ ]:
p = Point{Int64}(1, 2, 3.0)

In [ ]:
typeof(p.x)

In [ ]:
Int64 <: Real

In [ ]:
q1 = QVector([1, 0, 3.0])
q2 = QVector([1, 1, 2.0])
r = RVector([2.0, -3.0, 4.0])
qs = [q1, q2]
rs = Scalar([2.0, -3.0, 4.0])

In [ ]:
dot.(qs, rs)

In [ ]:
n = 1000
r1 = [2.0, -3.0, 4.0]
r2 = RVector([2.0, -3.0, 4.0])
rs = Scalar([2.0, -3.0, 4.0])
qs1 = [[rand(), rand(), rand()] for i in 1:n]
qs2 = rand(QVector, n)

function phase1()
    im * [dot(q, r1) for q in qs1]
end

function phase2()
    im * dot.(qs2, rs)
end

function phase3()
    im * [dot(q, r2) for q in qs2]
end

@btime phase1()
@btime phase2()
@btime phase3()
#     n = 1000
#     34.970 μs (1003 allocations: 39.33 KiB)
#     38.182 μs (1498 allocations: 47.13 KiB)
#     42.387 μs (2003 allocations: 70.58 KiB)

In [ ]:
Scalar(r2...)

In [ ]:
Scalar([r2...])

In [ ]:
phase(im)

## Testing Algorithm types

In [ ]:
abstract type AbstractAlgorithm end
abstract type Algorithm1D <:AbstractAlgorithm end
abstract type Algorithm2D <:AbstractAlgorithm end
abstract type Algorithm3D <:AbstractAlgorithm end
struct Trapz <: Algorithm1D end
struct GaussLegendre <: Algorithm1D end
struct GaussianProduct <: Algorithm2D
    x::AbstractAlgorithm # the algorithm used for x axis (or θ)
    y::AbstractAlgorithm # the algorithm used for y axis (or φ)
    GaussianProduct(x::AbstractAlgorithm=GaussLegendre(),y::AbstractAlgorithm=Trapz()) = new(x, y)
end

In [ ]:
gp = GaussianProduct()
gp.x, gp.y

In [ ]:
struct HCubature2D <: Algorithm2D
    kwargs::Dict
    HCubature2D(;kwargs::Dict=Dict()) = new(kwargs)
end

In [ ]:
hc = HCubature2D()

In [ ]:
hc.kwargs

In [ ]:
hc2 = HCubature2D(kwargs=Dict("atol"=>1e-5,"rtol"=>1e-6))

In [ ]:
hc2.kwargs

In [ ]:
function f1(x; k=1, t=2)
    x*k*t
end
kwargs=Dict(:k=>3,:t=>4)
f1(3), f1(3;kwargs...), f1(3;Dict()...)

In [ ]:
abstract type AbstractAlgorithm end
struct GeneralAlgorithm end

abstract type Algorithm{N} <: AbstractAlgorithm end
Algorithm1D = Algorithm{1}
Algorithm2D = Algorithm{2}
Algorithm3D = Algorithm{3}

struct Trapz <: Algorithm1D end
struct GaussLegendre <: Algorithm1D end
struct QuadGK <: Algorithm1D
    kwargs::Dict # extra arguments for QuandGK.quadgk
    QuadGK(;kwargs::Dict=Dict()) = new(kwargs)
end
    
struct GaussianProduct2D <: Algorithm2D
    x::Algorithm1D # the algorithm used for x axis (or θ)
    y::Algorithm1D # the algorithm used for y axis (or φ)
    GaussianProduct2D(x::Algorithm1D=GaussLegendre(),y::Algorithm1D=Trapz()) = new(x, y)
end

struct GaussianProduct3D <: Algorithm3D
    x::Algorithm1D # the algorithm used for x axis (or θ)
    y::Algorithm1D # the algorithm used for y axis (or φ)
    z::Algorithm1D # the algorithm used for z axis (or r)
    GaussianProduct3D(x::Algorithm1D=Trapz(),y::Algorithm1D=Trapz(),z::Algorithm1D=Trapz()) = new(x, y, z)
end

In [ ]:
function default(::GeneralAlgorithm, dim::Integer)
    if dim == 1
        algo = Trapz()
    elseif dim == 2
        algo = GaussianProduct2D()
    elseif dim == 3
        algo = GaussianProduct3D()
    end
end

In [ ]:
default(GeneralAlgorithm(), 3)

In [ ]:
tz = Trapz()
typeof(tz)

In [ ]:
tz isa Algorithm1D

In [ ]:
tz isa Algorithm2D

In [ ]:
tz isa AbstractAlgorithm

In [ ]:
tz isa Algorithm{1}

In [ ]:
gp2 = GaussianProduct2D()

In [ ]:
gp2 isa Algorithm2D

In [ ]:
gp2 isa Algorithm1D

In [ ]:
gp2 isa AbstractAlgorithm

In [ ]:
function size_avg(f::Function, a::SVector{N,T}, b::SVector{N,T}, d::Distribution; algo::Algorithm{N}=default(GeneralAlgorithm(),N)) where {N, T<:Real}
    println(algo)
    f(a[1])
end

d = Normal()
size_avg(cos, SVector{3}([0.0,0.0,0.0]), SVector{3}([1.0,1.0,1.0]), d)

In [ ]:
SVector{1}([1.0])

In [ ]:
nothing isa AbstractAlgorithm

In [ ]:
function alg(algo::Trapz)
    size_avg
end

func = alg(tz)

In [ ]:
size_avg(cos, SVector{3}([0.0,0.0,0.0]), SVector{3}([1.0,1.0,1.0]), d, algo=GaussianProduct3D(Trapz(),GaussLegendre(),Trapz()))

In [ ]:
gp2

In [ ]:
gp2.x.kwargs

In [ ]:
gp3 = GaussianProduct3D(QuadGK(),GaussLegendre(),Trapz())

In [ ]:
gp3.x.kwargs

In [ ]:
d3 = product_distribution([Normal(0,1),Normal(1,2),Normal(2,3)])

In [ ]:
pdf(d3, [0, 0, 0])

In [ ]:
SMatrix{1,1}(1.0)

In [ ]:
h = SMatrix{1,1}(2.0)

In [ ]:
h[1,1]

In [ ]:
typeof(h[1])

In [ ]:
SMatrix{3,3}(I)

## Testing Symmetry operators

In [ ]:
const RotMatrix{T} = SMatrix{3,3,T}
const Vector3D{T} = SVector{3,T}

In [ ]:
-RotMatrix(I)

In [ ]:
struct Inversion
    W::RotMatrix
    Inversion() = new(-RotMatrix(I))
end

In [ ]:
i = Inversion()

In [ ]:
i.W

In [ ]:
SVector{3}(zeros(3))

In [ ]:
struct Inverse
    W::RotMatrix
    w::Vector3D{Real}
    Inverse() = new(-RotMatrix(I), Vector3D(zeros(3)))
end

In [ ]:
i = Inverse()

In [ ]:
i.W

In [ ]:
i.w

In [ ]:
Integer <: Real

In [ ]:
typeof(1//2)

In [ ]:
Rational{Integer}

In [ ]:
typeof(Rational)

In [ ]:
inversion(n::Integer) = (-SMatrix{n,n}(I), SVector{n}(zeros(n)))

In [ ]:
inversion(2)

## Testing transform vectors from one space to another space

In [2]:
a = 2*[1.0, 0, 0]
b = 2*[-1/2, √3/2, 0]
c = [0.0, 0, 5]
Mi = transpose(hcat(a,b,c))

3×3 Transpose{Float64,Array{Float64,2}}:
  2.0  0.0      0.0
 -1.0  1.73205  0.0
  0.0  0.0      5.0

In [3]:
M = inv(Mi)

3×3 Transpose{Float64,Array{Float64,2}}:
 0.5       0.0      0.0
 0.288675  0.57735  0.0
 0.0       0.0      0.2

In [4]:
G = Mi * transpose(Mi)

3×3 Array{Float64,2}:
  4.0  -2.0   0.0
 -2.0   4.0   0.0
  0.0   0.0  25.0

In [5]:
Gi = inv(G)

3×3 Array{Float64,2}:
 0.333333  0.166667  -0.0 
 0.166667  0.333333  -0.0 
 0.0       0.0        0.04

In [6]:
abstract type AbstractSpace end
abstract type Space <: AbstractSpace end
abstract type Coordinate <: AbstractSpace end
struct RealSpace <: Space end
struct ReciprocalSpace <: Space end
# Cartesian coordinate system: basis vectors are orthonormal
struct CartesianSpace <: Coordinate end
# Basis vectors chosen according to 14 Bravais Lattices. It is often refered as fractional coordinate system.
struct BravaisSpace <: Coordinate end

In [7]:
struct CrystalSpace
    space::Space
    coord::Coordinate
end
CrystalSpace() = CrystalSpace(RealSpace(), CartesianSpace())

CrystalSpace

In [8]:
cs1 = CrystalSpace()
cs11 = CrystalSpace()
cs2 = CrystalSpace(RealSpace(), BravaisSpace())
cs3 = CrystalSpace(ReciprocalSpace(), CartesianSpace())
cs4 = CrystalSpace(ReciprocalSpace(), BravaisSpace())
@test cs1 == cs11
cs1 == cs3

false

In [9]:
cs1.space isa RealSpace

true

In [10]:
cs1.id, cs2.id, cs3.id, cs4.id

ErrorException: type CrystalSpace has no field id

In [11]:
function array3(fillval, ::Val{N}) where N
    fill(fillval, ntuple(d->3, Val(N)))
end

array3 (generic function with 1 method)

In [12]:
typeof(Val(2))

Val{2}

In [13]:
const RCSpace = cs1
const RBSpace = cs2
const QCSpace = cs3
const QBSpace = cs4

CrystalSpace(ReciprocalSpace(), BravaisSpace())

In [14]:
struct Shape end
function transform_matrix(shape::Shape, from::CrystalSpace, to::CrystalSpace)
    if from == to
        0
    elseif from == RCSpace && to == RBSpace
        1
    elseif from == RBSpace && to == RCSpace
        -1
    elseif from == RBSpace && to == QBSpace
        2
    elseif from == QBSpace && to == RBSpace
        -2
    elseif from == QBSpace && to == QCSpace
        3
    elseif from == QCSpace && to == QBSpace
        -3
    else
        throw("No transfromation matrix available from $from to $to.")
    end
end

transform_matrix (generic function with 1 method)

In [15]:
transform(v::SVector, shape::Shape, from::CrystalSpace, to::CrystalSpace) = transform_matrix(shape, from, to)

transform (generic function with 1 method)

In [16]:
transform_matrix(Shape(), cs3, cs3)

0

In [17]:
transform_matrix(Shape(), cs2, cs4)

2

In [18]:
transform_matrix(Shape(), cs1, cs4)

String: "No transfromation matrix available from CrystalSpace(RealSpace(), CartesianSpace()) to CrystalSpace(ReciprocalSpace(), BravaisSpace())."